In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("CRAG_grouped_0_1_2_with_retriever_rank_fixed.parquet")

In [3]:
df.columns

Index(['query', 'page_url', 'page_result', 'domain', 'question_type',
       'static_or_dynamic', 'answer', 'split', 'retriever_rank'],
      dtype='object')

In [4]:
df

,query,page_url,page_result,domain,question_type,static_or_dynamic,answer,split,retriever_rank
0,"Among among amazon, sephora and zoom, how many...",[https://companiesmarketcap.com/renewable-ener...,Largest renewable energy companies by market c...,finance,aggregation,slow-changing,"among the three, two companies, amazon and sep...",0,"{'distances': [0.7522778511047363, 0.760092020..."
1,abacus life's maximum price last week?,[https://www.nasdaq.com/market-activity/stocks...,Stock Market Activity Today & Latest Stock Mar...,finance,comparison,fast-changing,$25.05,0,"{'distances': [0.9911408424377441, 1.132053136..."
2,"are megan thee stallion's song ""hiss"" and nick...",[https://www.forbes.com/sites/conormurray/2024...,Megan Thee Stallion’s ‘Hiss’ Debuts At No. 1 O...,music,comparison,fast-changing,"no, nicki minaj's song ""big foot"" did not make...",0,"{'distances': [0.46255388855934143, 0.47322064..."
3,are you able to say the movie that won the awa...,[https://en.wikipedia.org/wiki/Academy_Award_f...,Academy Award for Best Documentary Feature Fil...,movie,simple,static,undefeated,0,"{'distances': [0.633389949798584, 0.7214317321..."
4,"as of now, what is the pe ratio of douyu inter...",[https://www.nasdaq.com/market-activity/stocks...,DouYu International Holdings Limited ADS (DOYU...,finance,simple,slow-changing,24.9,1,"{'distances': [0.426475465297699, 0.4740136563..."
...,...,...,...,...,...,...,...,...,...
895,why was freddie mercury's favorite instrument ...,[https://www.quora.com/Did-Freddie-Mercury-pla...,"Freddie Mercury - Biography, equipment and in...",music,false_premise,static,invalid question,1,"{'distances': [0.5641282796859741, 0.573102712..."
896,"with $1000 at yesterday's closing price, which...",[https://www.nerdwallet.com/reviews/investing/...,"Interactive Brokers Review 2024: Pros, Cons an...",finance,comparison,fast-changing,dhcni,0,"{'distances': [0.9369211196899414, 0.937952399..."
897,with what number of points did celta vigo play...,[https://www.sofascore.com/team/football/celta...,"Celta Vigo live score, schedule & player stats...",sports,false_premise,fast-changing,invalid question,0,"{'distances': [0.6282259225845337, 0.712649405..."
898,would you happen to know the price-to-earnings...,[https://www.nasdaq.com/market-activity/stocks...,Intel Corporation Common Stock (INTC) Price/Ea...,finance,simple,real-time,18.85,0,"{'distances': [0.5601609945297241, 0.702277660..."


In [9]:
lista = [x['texts'] for x in df.retriever_rank.values]

In [13]:
from collections import Counter

# lista[0] is a list of strings
lengths = [len(text) for text in lista]

# Count how many times each length appears
length_counts = Counter(lengths)

# Print each length and the count of elements with that length
for length, count in sorted(length_counts.items()):
    print(f"Length: {length}, Count: {count}")


Length: 3, Count: 900


In [ ]:
df.head(3)

In [ ]:
df.retriever_rank.values[0]

In [ ]:
import pandas as pd
df = pd.read_parquet("CRAG_grouped_0_1_2_with_retriever_rank_fixed.parquet")
df.columns
df.head(3)
df.retriever_rank.values[0]
import pandas as pd
import numpy as np
from itertools import permutations

def augment_retriever_ranks(df, num_permutations=5):
    augmented_rows = []

    for _, row in df.iterrows():
        rank = row['retriever_rank']
        triplet = list(zip(rank['distances'], rank['indices'], rank['texts']))
        
        # Generate all permutations except the original
        all_perms = list(permutations(triplet))
        original_tuple = tuple(triplet)
        all_perms.remove(original_tuple)

        # Select up to num_permutations unique permutations
        selected_perms = all_perms[:num_permutations]

        # Add the original row first
        augmented_rows.append(row.to_dict())

        # Add permuted rows
        for perm in selected_perms:
            distances, indices, texts = zip(*perm)
            new_rank = {
                'distances': np.array(distances),
                'indices': np.array(indices),
                'texts': np.array(texts)
            }
            new_row = row.to_dict()
            new_row['retriever_rank'] = new_rank
            augmented_rows.append(new_row)

    return pd.DataFrame(augmented_rows).reset_index(drop=True)

# Assuming df already has the retriever_rank column
df_augmented = augment_retriever_ranks(df, num_permutations=5)

df.shape
df_augmented.shape


In [ ]:
from RankThenStopEnv import RankThenStopEnv

In [ ]:
from RankThenStopEnv import RankThenStopEnv
from Functions.llm import get_evaluation_from_llm as reward_fn
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CheckpointCallback

# Env setup
env = RankThenStopEnv(df=df_augmented, reward_fn=reward_fn)

# # Callbacks
# checkpoint_callback = CheckpointCallback(save_freq=1000, save_path="./ppo_checkpoints/", name_prefix="ppo_model")

# # Train
# model = PPO("MultiInputPolicy", env, verbose=1, tensorboard_log="./ppo_tensorboard/")
# model.learn(total_timesteps=5000, callback=checkpoint_callback)

